### This notebook has various explorations of the market access tools, but is not desgined as a complete workflow. Use at your own discretion

In [34]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
from infrasap.misc import tPrint
#import infrasap.rai_calculator as rai

# Calculate OD matrix from origins and destinations

In [3]:
travel_surface_layer = "/home/wb411133/data/Country/NPL/OD_CALC/ct_monsoon_nozero2_4326_190305.tif"
origins_file = "/home/wb411133/data/Country/NPL/OD_CALC/Admin2_Headquarters_Origins.gpkg"
destinations_file = "/home/wb411133/data/Country/NPL/OD_CALC/Urban_Centers_Combined_4326.shp"

inR = rasterio.open(travel_surface_layer)
mcp = graph.MCP_Geometric(inR.read()[0,:,:])

In [22]:
origins = gpd.read_file(origins_file)
if origins.crs != inR.crs:
    origins = origins.to_crs(inR.crs)
destinations = gpd.read_file(destinations_file)
if destinations.crs != inR.crs:
    destinations = destinations.to_crs(inR.crs)

In [89]:
# Creates IDs for origins and destinations for attaching to the original files
dest_coords_full = list([inR.index(x.x, x.y) for x in origins['geometry']])
dest_coords_full = [f"{x[0]}_{x[1]}" for x in dest_coords_full]
origins['ID'] = dest_coords_full
origins['X'] = origins['ID'].apply(lambda x: int(x.split("_")[0]))
origins['Y'] = origins['ID'].apply(lambda x: int(x.split("_")[1]))

o_coords_full = list([inR.index(x.x, x.y) for x in destinations['geometry']])
o_coords_full = [f"{x[0]}_{x[1]}" for x in o_coords_full]
destinations['ID'] = o_coords_full
destinations['X'] = destinations['ID'].apply(lambda x: int(x.split("_")[0]))
destinations['Y'] = destinations['ID'].apply(lambda x: int(x.split("_")[1]))


In [92]:
print(origins['X'].min())
print(origins['X'].max())

print(destinations['X'].min())
print(destinations['X'].max())

print(origins['Y'].min())
print(origins['Y'].max())

print(destinations['Y'].min())
print(destinations['Y'].max())

1526
15425
1812
15268
130
25726
412
25832


In [59]:
# Identify destination coordinates ion the raster for calculating the OD matrix
#    NOTE - the length of the destinations may not match the length of the 
#           origins file due to coincident points
dest_coords = ma.get_mcp_dests(inR, origins)
dest_coords_id = [f"{x[0]}_{x[1]}" for x in dest_coords]

origin_coords = ma.get_mcp_dests(inR, destinations)
origin_coords_id = [f"{x[0]}_{x[1]}" for x in origin_coords]

results = np.zeros([len(dest_coords), len(origin_coords)])

In [75]:
print("***Origins")
print(origins.shape[0])
print(len(dest_coords_full))
print(len(dest_coords))
print("***Destinations")
print(destinations.shape[0])
print(len(o_coords_full))
print(len(origin_coords))

***Origins
754
754
753
***Destinations
508
508
501


In [36]:
# BEN TODO - I flipped the origins and destinations incorrectly.
#   Whichever of the two is smaller, that should be the destinations
for dIdx in range(0, len(dest_coords)):
    tPrint(f"Processing {dIdx} of {len(dest_coords)}")
    costs, traceback = mcp.find_costs([dest_coords[dIdx]])
    oIdx=0
    for o in origin_coords:
        res = costs[o]
        results[dIdx,oIdx] = res
        oIdx += 1    

16:16:49	Processing 0 of 753
16:19:36	Processing 1 of 753
16:22:27	Processing 2 of 753
16:25:16	Processing 3 of 753
16:28:04	Processing 4 of 753
16:30:42	Processing 5 of 753
16:33:19	Processing 6 of 753
16:33:21	Processing 7 of 753
16:36:17	Processing 8 of 753
16:38:57	Processing 9 of 753
16:41:42	Processing 10 of 753
16:44:21	Processing 11 of 753
16:47:13	Processing 12 of 753
16:49:59	Processing 13 of 753
16:52:55	Processing 14 of 753
16:55:47	Processing 15 of 753
16:58:22	Processing 16 of 753
17:01:09	Processing 17 of 753
17:03:50	Processing 18 of 753
17:06:28	Processing 19 of 753
17:09:17	Processing 20 of 753
17:12:05	Processing 21 of 753
17:14:45	Processing 22 of 753
17:17:33	Processing 23 of 753
17:20:18	Processing 24 of 753
17:23:06	Processing 25 of 753
17:25:57	Processing 26 of 753
17:28:53	Processing 27 of 753
17:31:34	Processing 28 of 753
17:34:30	Processing 29 of 753
17:37:15	Processing 30 of 753
17:39:54	Processing 31 of 753
17:42:42	Processing 32 of 753
17:45:23	Processing 

05:02:11	Processing 276 of 753
05:04:53	Processing 277 of 753
05:07:39	Processing 278 of 753
05:10:17	Processing 279 of 753
05:13:00	Processing 280 of 753
05:15:55	Processing 281 of 753
05:18:37	Processing 282 of 753
05:21:33	Processing 283 of 753
05:24:17	Processing 284 of 753
05:27:10	Processing 285 of 753
05:30:08	Processing 286 of 753
05:33:00	Processing 287 of 753
05:35:53	Processing 288 of 753
05:38:50	Processing 289 of 753
05:41:40	Processing 290 of 753
05:44:21	Processing 291 of 753
05:47:06	Processing 292 of 753
05:50:04	Processing 293 of 753
05:52:44	Processing 294 of 753
05:55:22	Processing 295 of 753
05:58:17	Processing 296 of 753
06:00:53	Processing 297 of 753
06:03:33	Processing 298 of 753
06:06:22	Processing 299 of 753
06:09:10	Processing 300 of 753
06:11:51	Processing 301 of 753
06:14:28	Processing 302 of 753
06:17:25	Processing 303 of 753
06:20:05	Processing 304 of 753
06:22:46	Processing 305 of 753
06:25:33	Processing 306 of 753
06:28:29	Processing 307 of 753
06:31:16

17:19:25	Processing 541 of 753
17:22:06	Processing 542 of 753
17:24:55	Processing 543 of 753
17:27:49	Processing 544 of 753
17:30:47	Processing 545 of 753
17:33:38	Processing 546 of 753
17:36:38	Processing 547 of 753
17:39:17	Processing 548 of 753
17:42:10	Processing 549 of 753
17:45:06	Processing 550 of 753
17:47:46	Processing 551 of 753
17:50:29	Processing 552 of 753
17:53:20	Processing 553 of 753
17:56:19	Processing 554 of 753
17:59:02	Processing 555 of 753
18:01:41	Processing 556 of 753
18:04:28	Processing 557 of 753
18:07:16	Processing 558 of 753
18:10:11	Processing 559 of 753
18:13:06	Processing 560 of 753
18:15:56	Processing 561 of 753
18:18:41	Processing 562 of 753
18:21:24	Processing 563 of 753
18:24:08	Processing 564 of 753
18:27:06	Processing 565 of 753
18:29:53	Processing 566 of 753
18:32:35	Processing 567 of 753
18:35:13	Processing 568 of 753
18:37:51	Processing 569 of 753
18:40:41	Processing 570 of 753
18:43:28	Processing 571 of 753
18:46:25	Processing 572 of 753
18:49:23

In [45]:
all_res = pd.DataFrame(results)
all_res.index = dest_coords_id
all_res.columns = origin_coords_id
all_res.to_csv("/home/wb411133/temp/NPL_OD.csv")

(753, 501)

In [95]:
all_res

,15154_23725,10632_13709,14925_23064,10887_14939,10180_13924,9363_13945,13230_18740,14494_21863,13473_16286,15205_25567,...,4956_4939,14056_22741,11672_22820,14461_24737,9673_12016,14128_24277,12569_15214,13879_19276,14437_24577,9939_16555
9398_16084,0.366462,0.130592,0.334874,0.117654,0.138797,0.121938,0.213074,0.271128,0.192627,0.462613,...,0.528237,0.299209,0.345271,0.428848,0.196511,0.410673,0.165158,0.229612,0.418748,0.061013
11376_13609,0.324553,0.033170,0.292965,0.043449,0.065199,0.099319,0.171166,0.229219,0.126304,0.420704,...,0.454639,0.257300,0.303362,0.386939,0.122913,0.368764,0.080473,0.187703,0.376839,0.161158
12803_15018,0.310707,0.099307,0.279119,0.067362,0.128998,0.163118,0.157320,0.215374,0.058645,0.406858,...,0.518438,0.243455,0.289516,0.373093,0.186712,0.354918,0.018171,0.173857,0.362993,0.168026
6597_4647,0.657179,0.344306,0.625590,0.376075,0.346917,0.356086,0.503791,0.561845,0.458930,0.753329,...,0.153939,0.589926,0.635987,0.719564,0.275989,0.701390,0.417514,0.520329,0.709465,0.448469
15425_23278,0.038289,0.318851,0.030283,0.286906,0.348542,0.382662,0.189121,0.113868,0.292372,0.134439,...,0.737982,0.069956,0.147838,0.100674,0.406256,0.082499,0.290587,0.177515,0.090574,0.326568
11351_24078,0.218837,0.387186,0.187248,0.355240,0.416876,0.450996,0.257455,0.182202,0.360706,0.269576,...,0.806316,0.170111,0.197462,0.241635,0.474590,0.209284,0.358921,0.245849,0.243271,0.394902
10482_7261,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7979_9691,0.505861,0.192988,0.474272,0.224757,0.195600,0.204768,0.352473,0.410527,0.307612,0.602012,...,0.309821,0.438608,0.484669,0.568246,0.124671,0.550072,0.266196,0.369011,0.558147,0.297151
2493_5883,0.842312,0.529439,0.810724,0.561209,0.532051,0.541219,0.688925,0.746978,0.644063,0.938463,...,0.127679,0.775060,0.821121,0.904698,0.461122,0.886523,0.602648,0.705462,0.894598,0.633603
9948_17605,0.289113,0.177496,0.257525,0.155362,0.185701,0.168842,0.135726,0.193780,0.163870,0.385264,...,0.575141,0.221861,0.267922,0.351499,0.243415,0.333324,0.159043,0.152263,0.341399,0.084238


In [70]:
origins.to_file("/home/wb411133/temp/NPL_OD_origins.geojson", driver="GeoJSON")
destinations.to_file("/home/wb411133/temp/NPL_OD_destinations.geojson", driver="GeoJSON")

# Calculate market sheds from travel datasets
https://scikit-image.org/docs/dev/api/skimage.graph.html#skimage.graph.MCP_Geometric


In [ ]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

In [ ]:
tutorial_folder ="../tutorial_data"
img = os.path.join(tutorial_folder, "global_friction_surface.tif")
dests = os.path.join(tutorial_folder, "destinations.shp")

img = rasterio.open(img)
inD = gpd.read_file(dests)
data = img.read()[0,:,:]

In [ ]:
mcp = graph.MCP_Geometric(data)
dests_geom = [img.index(x.x, x.y) for x in inD['geometry']]
costs, traceback = mcp.find_costs(dests_geom)

In [ ]:
meta = img.meta.copy()
meta.update(dtype=costs.dtype)

with rasterio.open(os.path.join(tutorial_folder, "market_shed_custom.tif"), 'w', **meta) as outR:
    outR.write_band(1, costs)

# Debugging below

In [ ]:
import copy
# testing market sheds
dests_geom = [img.index(x.x, x.y) for x in inD['geometry']]
all_c = []
n = inD.shape[0]
idx = 0

In [ ]:
for dest in dests_geom:
    idx += 1
    if dest[0] > 0 and dest[0] < img.shape[0] and dest[1] > 0 and dest[1] < img.shape[1]:
        c1, trace = mcp.find_costs([dest])
        all_c.append(copy.deepcopy(c1))
    else:
        print(f"{idx} of {n} cannot be processed")


In [ ]:
# Iterate through results to generate final marketshed
res = np.zeros(all_c[0].shape)
for idx in range(0, len(all_c)):
    cur_res = all_c[idx]
    if idx == 0:
        min_res = cur_res
    else:
        combo = np.dstack([min_res, cur_res])
        min_res = np.amin(combo, 2)
        cur_val = (min_res == cur_res).astype(np.byte)
        m_idx = np.where(cur_val == 1)
        res[m_idx] = idx


In [ ]:
meta = img.meta.copy()
meta.update(dtype=res.dtype)
with rasterio.open(os.path.join(tutorial_folder, "market_shed_custom.tif"), 'w', **meta) as outR:
    outR.write_band(1, res)

In [ ]:
output.shape

In [ ]:
class MarketShedMCP(graph.MCP_Flexible):
    def _reset(self):
        """reset the id map
        """
        graph.MCP_Flexible._reset(self)
        self._conn = {}
        self._bestconn_v = {}
        self._bestconn = {}
    
    def create_connection(self, id1, id2, pos1, pos2, cost1, cost2):
        # Process data
        hash = min(id1, id2), max(id1, id2)
        val = min(pos1, pos2), max(pos1, pos2)
        cost = min(cost1, cost2)
        # Add to total list 
        self._conn.setdefault(hash, []).append(val)
        # Keep track of connection with lowest cost
        curcost = self._bestconn_v.get(hash, (np.inf,))[0]        
        if cost1 < cost2:
            self._bestconn[val] = (id1,)
        else:
            self._bestconn[val] = (id2,)        
        if cost < curcost:            
            self._bestconn_v[hash] = (cost,) + val
    
mcp_m = MarketShedMCP(data)
costs, traceback = mcp_m.find_costs(dests_geom)

In [ ]:
data.shape[0] * data.shape[1]

In [ ]:
len(mcp_m._bestconn.keys())

In [ ]:
graph.MCP_Flexible.update_node?

In [ ]:
costs, traceback = mcp.find_costs(dests_geom, find_all_ends=True)

In [ ]:
meta = img.meta.copy()
meta.update(dtype=res.dtype, count=5)
with rasterio.open(os.path.join(tutorial_folder, "market_shed.tif"), 'w', **meta) as outR:
    for idx in range(0,5):
        outR.write_band(idx + 1, res[:,:,idx])

In [ ]:
def get_min_axis(x):
    return(np.where(x == x.min()))

res_min = np.apply_along_axis(get_min_axis, 2, res)

In [ ]:
meta = img.meta.copy()
res_min = res_min.astype(meta['dtype'])

In [ ]:
res_min.shape

In [ ]:

meta.update(dtype=res_min.dtype)
with rasterio.open(os.path.join(tutorial_folder, "market_shed_2.tif"), 'w', **meta) as outR:
    outR.write_band(1, res_min[:,:,0,0])

'''
meta.update(dtype=costs.dtype)
with rasterio.open(os.path.join(tutorial_folder, "travel_costs_fa.tif"), 'w', **meta) as outR:
    outR.write_band(1, costs)
    
meta.update(dtype=traceback.dtype)
with rasterio.open(os.path.join(tutorial_folder, "traceback_fa.tif"), 'w', **meta) as outR:
    outR.write_band(1, traceback)
'''

In [ ]:
sys.path.append("../")

import infrasap.market_access as ma

In [ ]:
importlib.reload(ma)
out_file = os.path.join(tutorial_folder, "market_shed_3.tif")
ma.generate_market_sheds(img, mcp, inD, out_file)

# Calculate area of cells in raster dataset

In [ ]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"

inG = rasterio.open(global_friction_surface)
inD = inG.read()

In [ ]:
inD.shape

In [ ]:
inG.bounds

In [ ]:
def generate_shape(x, llx, lly, res):
    lly = lly + (x * res)
    ll = (llx, lly)
    ul = (llx, lly + res)
    ur = (llx + res, lly + res)
    lr = (llx + res, lly)
    shape = Polygon([ll, ul, ur, lr, ll])
    return(shape)
    
b = inG.bounds
res = inG.meta['transform'][0]
all_shapes = [generate_shape(x, b[0], b[1], res) for x in range(0, inD.shape[1])]

In [ ]:
res = pd.DataFrame(columns=["idx"])
res['idx'] = list(range(0, inG.shape[0]))
res = gpd.GeoDataFrame(res, geometry=all_shapes, crs = inG.crs)
res['area'] = res['geometry'].apply(lambda x: x.area)

In [ ]:
res.to_file("/home/wb411133/temp/gfs_column1.shp")

In [ ]:
# Convert to UTM to calculate area to metres
'''sys.path.append("../../GOST")

import GOSTRocks.misc as misc
res_sel = res.iloc[1:-1,]
res_utm = misc.project_UTM(res_sel)'''

res_utm = res.to_crs({'init':'epsg:32601'})

In [ ]:
res_utm['area_utm'] = res_utm['geometry'].apply(lambda x: x.area)
res_utm.to_file("/home/wb411133/temp/gfs_column1.shp")

In [ ]:
res_utm.head()

In [ ]:
import numpy as np
import skimage.graph as graph
import copy

In [ ]:
img = np.array([[1,1,2,2,2],[2,1,1,3,3],[3,2,1,2,3],[2,2,2,1,1]])
mcp = graph.MCP_Geometric(img)

In [ ]:
destinations = [[0,0],[3,3]]
costs, traceback = mcp.find_costs(destinations)
print(costs)
print(traceback)

In [ ]:
output

In [ ]:
all_c = []
for dest in destinations:
    costs, traceback = mcp.find_costs([dest])
    all_c.append(copy.deepcopy(costs))

In [ ]:
res = np.dstack(all_c)
res_min = np.amin(res, axis=2)
output = np.zeros([res_min.shape[0], res_min.shape[1]])
for idx in range(0, res.shape[2]):
    cur_data = res[:,:,idx]
    cur_val = (cur_data == res_min).astype(np.byte) * idx
    output = output + cur_val
output = output.astype(np.byte)


In [ ]:
output

In [ ]:
res[:,:,1]

In [ ]:
class MarketShedMCP(graph.MCP_Geometric):
    def _reset(self):
        """reset the id map
        """
        graph.MCP_Flexible._reset(self)
        self._conn = {}
        self._bestconn_v = {}
        self._bestconn = {}
    
    def create_connection(self, id1, id2, pos1, pos2, cost1, cost2):
        # Process data
        hash = min(id1, id2), max(id1, id2)
        print(hash)
        val = min(pos1, pos2), max(pos1, pos2)
        cost = min(cost1, cost2)
        # Add to total list 
        self._conn.setdefault(hash, []).append(val)
        # Keep track of connection with lowest cost
        curcost = self._bestconn_v.get(hash, (np.inf,))[0]        
        if cost1 < cost2:
            self._bestconn[val] = (id1,)
        else:
            self._bestconn[val] = (id2,)        
        if cost < curcost:            
            self._bestconn_v[hash] = (cost,) + val
    
mcp_m = MarketShedMCP(img)
costs, traceback = mcp_m.find_costs(destinations)

In [ ]:
costs

In [ ]:
import numpy as np
from skimage import graph

image = np.array(
    [[1, 1, 2, 2, 2, 2],
     [2, 1, 1, 3, 3, 3],
     [3, 2, 1, 2, 2, 2],
     [3, 2, 2, 1, 1, 1],
     [4, 3, 2, 1, 1, 4],
     [4, 3, 2, 1, 1, 4]]
)
destinations = [[0, 0], [3, 3]]
mcp = graph.MCP_Geometric(image)
costs, traceback = mcp.find_costs(destinations)

In [ ]:
offsets = _mcp.make_offsets(2, True)
offsets.append(np.array([0, 0]))
offsets_arr = np.array(offsets)
indices = np.indices(traceback.shape)
offset_to_neighbor = offsets_arr[traceback]
neighbor_index = indices - offset_to_neighbor.transpose((2, 0, 1))
ids = np.arange(traceback.size).reshape(costs.shape)
neighbor_ids = np.ravel_multi_index(
    tuple(neighbor_index), traceback.shape
)
g = sparse.coo_matrix((
        np.ones(traceback.size),
        (ids.flat, neighbor_ids.flat)),
    )
g.toarray().shape




In [ ]:
sparse.coo_matrix?

In [ ]:
n, components = sparse.csgraph.connected_components(g, directed=False)
basins = components.reshape(costs.shape)

In [ ]:
n, components = sparse.csgraph.connected_components(ids, directed=False)
basins = components.reshape(costs.shape)

In [ ]:
neighbor_ids